In [37]:
import pandas as pd
from git import Repo
import os
import json

In [6]:
# Find the parent directory of the repo
repo = Repo(os.getcwd(), search_parent_directories=True)
repo_root = repo.git.rev_parse("--show-toplevel")

# Construct the path to the data file
data_path = os.path.join(repo_root, 'data', 'DFT_data.csv')
print(data_path)

/home/mudaliar.k/github/Comformer_uv/data/DFT_data.csv


In [44]:
split_path = os.path.join(repo_root, "output/D2R2_multi3/ids_train_val_test.json")
with open(split_path, 'r') as f:
    train_test_val = json.loads(f.read())

len(train_test_val['id_test'][:10])

10

In [7]:
results_path = os.path.join(repo_root,"output/D2R2_multi3/multi_out_predictions.json")
print(results_path)

/home/mudaliar.k/github/Comformer_uv/output/D2R2_multi3/multi_out_predictions.json


In [15]:
results_df = pd.read_json(results_path)
data_df = pd.read_csv(data_path)

In [16]:
data_df['id'] = data_df["mpid"].astype(str) + data_df["miller"].astype(str) + data_df["term"].astype(str)

In [45]:
test_data_df = data_df[data_df['id'].isin(train_test_val['id_test'])]

In [17]:
results_df.head()

,id,target,predictions
0,mp-5615m1010,"[0.12365421652793801, 0.105956315994262, -1.34...","[0.10867144167423201, 0.11597524583339601, -1...."
1,mp-1079698m1011,"[0.990391492843627, 0.985259175300598, -1.3461...","[0.9636493325233461, 0.9841752052307121, -1.34..."
2,mp-867870m1100,"[0.8482617139816281, 0.38608622550964306, -1.3...","[0.8693876266479491, 0.41131627559661804, -1.3..."
3,mp-19746m1000,"[1.270712018013, 0.34578686952590904, -1.32017...","[1.171862363815307, 0.478311300277709, -1.3286..."
4,mp-1067587m1011,"[0.8209231495857231, 1.5089854001998901, -1.27...","[1.257728099822998, 0.9589100480079651, -1.291..."


In [18]:
data_df.head()

,Unnamed: 0,mpid,miller,term,energy,bulk_energy,convergence,Fermi,slab,nsites,...,sym,sym_vac,broken_bonds,area,opt_tol,max_normal_search,cleavage_energy,WF_bottom,WF_top,id
0,1,mp-1058581,m111,0,-15294.75699,-15298.816080,550-4-4-1,1.7996,"{'@module': 'pymatgen.core.structure', '@class...",16,...,True,True,14,74.984486,1.0,2,0.027066,2.301425,2.287049,mp-1058581m1110
1,3,mp-1058581,m110,0,-15296.12141,-15298.840520,550-4-4-1,2.2986,"{'@module': 'pymatgen.core.structure', '@class...",16,...,True,True,8,66.538369,0.1,3,0.020433,2.338018,2.360218,mp-1058581m1100
2,5,mp-1058581,m101,0,-15295.70542,-7649.382492,550-4-4-1,2.4416,"{'@module': 'pymatgen.core.structure', '@class...",16,...,True,True,10,67.756083,0.1,1,0.022578,2.312259,2.310812,mp-1058581m1010
3,7,mp-1058581,m100,0,-11471.47077,-3824.709179,550-4-4-1,1.9164,"{'@module': 'pymatgen.core.structure', '@class...",12,...,True,True,8,58.271504,0.1,0,0.022796,2.324314,2.317646,mp-1058581m1000
4,9,mp-1058581,m011,0,-15296.59636,-7649.397502,550-6-2-1,2.7173,"{'@module': 'pymatgen.core.structure', '@class...",16,...,True,True,5,47.192212,0.1,1,0.023295,2.347947,2.380204,mp-1058581m0110


In [ ]:
results_df[results_df['id'] == 'mp-1079698m1011']


,id,target,predictions
1,mp-1079698m1011,"[0.990391492843627, 0.985259175300598, -1.3461...","[0.9636493325233461, 0.9841752052307121, -1.34..."


In [48]:
target_columns = ["WF_bottom", "WF_top", "cleavage_energy"]
predicted_columns = ["WF_bottom_predicted", "WF_top_predicted", "cleavage_energy_predicted"]

# Convert lists in 'target' and 'predictions' columns into separate columns
results_df[target_columns] = results_df['target'].apply(lambda x: pd.Series(x))
results_df[predicted_columns] = results_df['predictions'].apply(lambda x: pd.Series(x))

# Drop the original 'target' and 'predictions' columns if no longer needed
results_df.drop(columns=['target', 'predictions'], inplace=True)
results_df.head()

,id,WF_bottom,WF_top,cleavage_energy,WF_bottom_predicted,WF_top_predicted,cleavage_energy_predicted
0,mp-5615m1010,0.123654,0.105956,-1.343176,0.108671,0.115975,-1.339667
1,mp-1079698m1011,0.990391,0.985259,-1.346190,0.963649,0.984175,-1.347725
2,mp-867870m1100,0.848262,0.386086,-1.325548,0.869388,0.411316,-1.324540
3,mp-19746m1000,1.270712,0.345787,-1.320170,1.171862,0.478311,-1.328629
4,mp-1067587m1011,0.820923,1.508985,-1.277064,1.257728,0.958910,-1.291403


In [47]:
test_data_df.describe()

,Unnamed: 0,term,energy,bulk_energy,Fermi,nsites,thickness_n,thickness_A,nterm,broken_bonds,area,opt_tol,max_normal_search,cleavage_energy,WF_bottom,WF_top
count,3685.000000,3685.000000,3685.000000,3685.000000,3685.000000,3685.000000,3685.000000,3685.000000,3685.000000,3685.000000,3685.000000,3685.000000,3685.000000,3685.000000,3685.000000,3685.000000
mean,28109.343555,1.736771,-102136.691119,-75545.097647,4.870374,32.950339,1.802714,10.501681,5.165265,7.452917,43.141605,0.462849,1.618453,0.099508,3.894086,3.925767
std,15683.562775,3.469530,69022.379288,69015.840687,1.714545,15.417867,0.881646,5.664575,5.506199,4.533417,18.863023,0.371442,1.203454,0.046780,0.856679,0.875034
min,15.000000,0.000000,-433585.239500,-393427.720900,-2.610100,6.000000,1.000000,1.379360,1.000000,0.000000,6.808774,0.100000,0.000000,0.000314,1.642623,1.641550
25%,14685.000000,0.000000,-141262.405300,-101768.301300,3.810600,20.000000,1.000000,5.817422,2.000000,4.000000,28.080417,0.100000,0.000000,0.070932,3.342088,3.354764
50%,28611.000000,1.000000,-84887.717400,-50324.317530,4.905600,32.000000,2.000000,10.931069,4.000000,7.000000,43.179152,0.500000,2.000000,0.088007,3.861841,3.858189
75%,41737.000000,2.000000,-49495.494230,-25632.641980,6.000900,40.000000,2.000000,14.709545,5.000000,10.000000,57.943531,1.000000,3.000000,0.119583,4.353667,4.387579
max,54954.000000,29.000000,-2823.899584,-1136.877649,11.672800,80.000000,5.000000,25.009676,32.000000,26.000000,180.797965,1.000000,3.000000,0.396299,10.601487,10.494136


In [46]:
test_data_df[test_data_df['id'] == 'mp-1079698m1011'][["WF_bottom", "WF_top", "cleavage_energy"]]

,WF_bottom,WF_top,cleavage_energy
16307,4.555698,4.545802,0.05046


In [49]:
selected_columns = ["WF_bottom", "WF_top", "cleavage_energy"]

# Compute mean and standard deviation
mean_values = test_data_df[selected_columns].mean()
std_values = test_data_df[selected_columns].std()

mean_values, std_values

(WF_bottom          3.894086
 WF_top             3.925767
 cleavage_energy    0.099508
 dtype: float64,
 WF_bottom          0.856679
 WF_top             0.875034
 cleavage_energy    0.046780
 dtype: float64)

In [50]:
# De-standardize the true columns
results_df['WF_bottom'] = results_df['WF_bottom'] * std_values['WF_bottom'] + mean_values['WF_bottom']
results_df['WF_top'] = results_df['WF_top'] * std_values['WF_top'] + mean_values['WF_top']
results_df['cleavage_energy'] = results_df['cleavage_energy'] * std_values['cleavage_energy'] + mean_values['cleavage_energy']

# De-standardize the predicted columns
results_df['WF_bottom_predicted'] = results_df['WF_bottom_predicted'] * std_values['WF_bottom'] + mean_values['WF_bottom']
results_df['WF_top_predicted'] = results_df['WF_top_predicted'] * std_values['WF_top'] + mean_values['WF_top']
results_df['cleavage_energy_predicted'] = results_df['cleavage_energy_predicted'] * std_values['cleavage_energy'] + mean_values['cleavage_energy']

results_df.head()

,id,WF_bottom,WF_top,cleavage_energy,WF_bottom_predicted,WF_top_predicted,cleavage_energy_predicted
0,mp-5615m1010,4.000018,4.018483,0.036674,3.987183,4.027250,0.036838
1,mp-1079698m1011,4.742534,4.787902,0.036533,4.719624,4.786954,0.036461
2,mp-867870m1100,4.620774,4.263606,0.037498,4.638872,4.285683,0.037546
3,mp-19746m1000,4.982678,4.228343,0.037750,4.897996,4.344306,0.037354
4,mp-1067587m1011,4.597354,5.246181,0.039767,4.971555,4.764846,0.039096


In [51]:
test_data_df[test_data_df['id'] == 'mp-5615m1010']

,Unnamed: 0,mpid,miller,term,energy,bulk_energy,convergence,Fermi,slab,nsites,...,sym,sym_vac,broken_bonds,area,opt_tol,max_normal_search,cleavage_energy,WF_bottom,WF_top,id
10160,16271,mp-5615,m101,0,-50320.4581,-50326.67087,550-4-4-1,2.8052,"{'@module': 'pymatgen.core.structure', '@class...",27,...,True,False,13,55.20331,0.5,2,0.056272,2.884514,2.85039,mp-5615m1010
